<a href="https://colab.research.google.com/github/chakiAunkit/AIBlogPostSummarizer/blob/master/AIBlogPostSummarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

## 1. Installing and importing libraries

In [3]:
!pip install transformers

     |████████████████████████████████| 4.2 MB 12.8 MB/s 
     |████████████████████████████████| 84 kB 3.0 MB/s 
     |████████████████████████████████| 6.6 MB 38.6 MB/s 
     |████████████████████████████████| 596 kB 48.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
from transformers import pipeline
from bs4 import BeautifulSoup
import requests

## 2. Load Summarization pipeline from HuggingFace

In [5]:
summarizer = pipeline('summarization')

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 (https://huggingface.co/sshleifer/distilbart-cnn-12-6)


Downloading:   0%|          | 0.00/1.76k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

## 3. Scrape the blog post from internet.

In [7]:
URL = "https://hackernoon.com/top-6-crypto-launchpads-in-2021-3fh34au"

In [11]:
r = requests.get(URL)
#print(r.text)

In [12]:
soup = BeautifulSoup(r.text, 'html.parser')
results = soup.find_all(['h1', 'p'])

In [17]:
#results

In [20]:
results[9]

<p class="paragraph">Binance is the leading cryptocurrency exchange and fastest-growing trading platform based on trading volumes. Launched in 2017, Binance, as a platform, has not limited itself to just being a cryptocurrency exchange. Instead, it has expanded into a complete ecosystem for all kinds of crypto services as it establishes a strong foothold in the fastest-growing sector. </p>

In [23]:
text = [result.text for result in results]
article = ' '.join(text)

In [26]:
#text

In [27]:
article

"Top 6 Crypto Launchpads in 2021 👋 I write fluff-free & engaging content in crypto space, ECDSA to zk-SNARKs, Bitcoin to Polkadot, DeFi to NFTs 👨\u200d💻 There is one universal truth in the crypto space - whoever gets in early, wins! Crypto aggregators list projects when they get significant amounts of traction. Discovering and identifying projects when they are at an early stage is very difficult. Missing out on those projects means you're leaving out massive returns on the table.\xa0 Is there a platform or an aggregator to discover early-stage crypto projects before they get to the mainstream? Absolutely, and we call them Crypto Launchpads. Think of these platforms as AngelList crypto.\xa0 Crypto launchpads have been gaining tremendous amounts of attention. They allow you to identify early-stage crypto projects and participate in their presale rounds. They also have a vetting process in place to avoid scams and rug-pulls, so you can invest with peace of mind without worrying about los

## 4. Chunking the processed text

In [28]:
article = article.replace('.', '.<eos>')
article = article.replace('!', '!<eos>')
article = article.replace('?', '?<eos>')

sentences = article.split('<eos>')

In [31]:
#sentences

In [32]:
sentences[7]

'\xa0 Crypto launchpads have been gaining tremendous amounts of attention.'

In [33]:
max_chunk = 500
current_chunk = 0
chunks = []

for sentence in sentences:
  if len(chunks) == current_chunk+1:
    if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
      chunks[current_chunk].extend(sentence.split(' '))
    else:
      current_chunk += 1;
      chunks.append(sentence.split(' '))
  
  else:
    print(current_chunk)
    chunks.append(sentence.split(' '))

0


In [37]:
#chunks[0]

In [38]:
for chunk_id in range(len(chunks)):
  chunks[chunk_id] = ' '.join(chunks[chunk_id])

In [41]:
len(chunks[3].split(' '))

480

In [44]:
chunks[2]

" Red Kite launchpad has implemented a lane-based swap system that allows each participant to get an opportunity to join based on their tier. \xa0 Integrated vesting schedule - Unlike other launchpads, Red Kite launchpad has a distribution portal that will allow projects to vest their tokens in pre-sale or post-sale secondary offerings. \xa0 Parachain Crowdloan - Red Kite launchpad will fully support crowdloan campaigns to secure parachain slots on Kusama and Polkadot.  Projects will be able to collect DOT or KSM tokens to win the parachain auction.  Red Kite is part of the PolkaFoundry ecosystem and supports the platform's native PKF token as a utility token.  Investors have to hold PKF tokens to participate in the token sale rounds on the Red Kite launchpad. \xa0 Red Kite launchpad now fully supports Coinbase Wallet and BSC Wallet and has integrated Blockpass to secure the KYC process.  This is just the beginning for Red Kite, and it aims to evolve from just being a launchpad to offe

## 5. Summarize text

In [61]:
res = summarizer(chunks, max_length=120, min_length=20, do_sample=False)

In [62]:
res[0].keys()

dict_keys(['summary_text'])

In [63]:
res

[{'summary_text': ' Crypto launchpads have been gaining tremendous amounts of attention . They allow you to identify early-stage crypto projects and participate in presale rounds . They also have a vetting process in place to avoid scams and rug-pulls . Binance Launchpad is known as the first successful example of a crypto launchpad .'},
 {'summary_text': " Red Kite is a platform where they launch hand-picked projects and allow investors to participate in a transparent and assured token sale process . RedKite is backed by the community, influencers, and marketing partners to help your project get in front of the investors . The platform has a tier-based reputation system that monitors each participant's behavior and assigns reputation points ."},
 {'summary_text': ' Red Kite launchpad has implemented a lane-based swap system that allows each participant to get an opportunity to join based on their tier . Parachain Crowdloan campaigns to secure parachain slots on Kusama and Polkadot wil

In [64]:
res[0]

{'summary_text': ' Crypto launchpads have been gaining tremendous amounts of attention . They allow you to identify early-stage crypto projects and participate in presale rounds . They also have a vetting process in place to avoid scams and rug-pulls . Binance Launchpad is known as the first successful example of a crypto launchpad .'}

In [65]:
' '.join([sum['summary_text'] for sum in res])

" Crypto launchpads have been gaining tremendous amounts of attention . They allow you to identify early-stage crypto projects and participate in presale rounds . They also have a vetting process in place to avoid scams and rug-pulls . Binance Launchpad is known as the first successful example of a crypto launchpad .  Red Kite is a platform where they launch hand-picked projects and allow investors to participate in a transparent and assured token sale process . RedKite is backed by the community, influencers, and marketing partners to help your project get in front of the investors . The platform has a tier-based reputation system that monitors each participant's behavior and assigns reputation points .  Red Kite launchpad has implemented a lane-based swap system that allows each participant to get an opportunity to join based on their tier . Parachain Crowdloan campaigns to secure parachain slots on Kusama and Polkadot will be able to collect DOT or KSM tokens to win the parachain au

## 6. Output to a .txt file

In [67]:
content = ' '.join([sum['summary_text'] for sum in res])

In [68]:
with open('summary.txt', 'w') as f:
  f.write(content)